## Import

In [1]:
import os
import sys
import argparse
import numpy as np
import supervision as sv
from pathlib import Path
from ultralytics import YOLO
from dotenv import load_dotenv

## Path & Env

In [2]:
# path
current_path = Path(os.getcwd())
sys.path.insert(0, current_path)

In [13]:
current_path

PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-8-56-352-16-jh/code/Users/jeremy.heng/opencv_yolo_supervision/script')

In [3]:
# env
dotenv_path = f"{current_path}/.env"
load_dotenv(dotenv_path=dotenv_path, override=True) 

False

## Model

In [4]:
model = YOLO("yolov8n-seg.pt")
mask_annotator = sv.MaskAnnotator()
label_annotator = sv.LabelAnnotator()
tracker = sv.ByteTrack()

100%|██████████| 6.74M/6.74M [00:00<00:00, 212MB/s]


In [5]:
# check object class ID
print(model.model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

## Annotation

In [6]:
def process_frame(frame: np.ndarray, frame_index: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # filter person obj
    person_class_id = 0 
    detections = detections[detections.class_id == person_class_id]

    # ID tracking
    detections = tracker.update_with_detections(detections)

    # label with class name + ID
    labels = []
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id] if class_id is not None else "unknown"
        label = f"{class_name} #{tracker_id}" if tracker_id is not None else class_name
        labels.append(label)

    # draw segmentation
    annotated_frame = mask_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )

    # draw labels
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels
    )

    return annotated_frame

## Main Process

In [14]:
def main(source_video_path: str):
    target_video_path = f"{current_path.parent}/result/result_segmentation.mp4"
    print(f"Processing video from '{source_video_path}' and saving to '{target_video_path}'...")
    sv.process_video(
        source_path=source_video_path,
        target_path=target_video_path,
        callback=process_frame
    )
    print("Video processing complete.")

In [16]:
if __name__ == "__main__":
    video_path = f"{current_path.parent}/videos/finance_20251001_1000_1001.mp4"
    main(video_path)

Processing video from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-8-56-352-16-jh/code/Users/jeremy.heng/opencv_yolo_supervision/videos/finance_20251001_1000_1001.mp4' and saving to '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-8-56-352-16-jh/code/Users/jeremy.heng/opencv_yolo_supervision/result_segmentation.mp4'...

0: 384x640 5 persons, 2 bottles, 2 cups, 3 chairs, 41.0ms
Speed: 3.0ms preprocess, 41.0ms inference, 505.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 bottles, 1 cup, 3 chairs, 1 tv, 7.6ms
Speed: 1.8ms preprocess, 7.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 bottles, 1 cup, 3 chairs, 1 tv, 7.5ms
Speed: 1.8ms preprocess, 7.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 2 bottles, 1 cup, 3 chairs, 1 tv, 7.5ms
Speed: 1.8ms preprocess, 7.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 bottles